In [1]:
# !pip install datasets transformers evaluate

In [1]:
from datasets import load_dataset

C:\Users\modaj\anaconda3\envs\DL-torch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import numpy as np
import pandas as pd

## `data`

In [19]:
QA_df = pd.read_csv("../data/moh_QA_clean.csv")

In [20]:
QA_df 

,question_id,question_description,answer_category_num,question_description_clean
0,1,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,Religious affiliation,وزير خارجي لبناني جبران باسيل سلسله تغريد عقب ...
1,2,سورية بلد الحضارات تربطها بعلية او بحيوان,Violent,سوري بلد حضاره ربط عليه حيوان
2,4246,تقتلون وسام الحسن وتترحموعلية من أي أصناف المخ...,Racist,قتل وسام حسن وتترحموعليه اي صنف مخلوق انتم
3,5304,معك خبر انو بلدة قطر متل ما سميتا مساحتها اكبر...,Normal,معك نوي بلد قطر متل سمي مساحه اكبر لبنان ل عيب...
4,1706,للامانه قوت الموسم اللي طاف كان هوا بس متحمس ح...,Normal,امانه قوت موسم ل ي طاف هوا متحمس حق الجاي ان ا...
...,...,...,...,...
3132,3720,كلامك هراء من دون اي قيمة تذكر انت بلوة من الب...,Violent,كلام هراء قيمه ذكر بلا البلوات منتقل قتل قتيل ...
3133,7784,راح خبرك شو شايفة ب جبران باسيل,Normal,خبر شو شايفه جبران باسيل
3134,1318,كلمة جبران باسيل أخجلته وأخجلت كل الأمة العربية,Sexual harrasment,كلمه جبران باسيل اخجل اخجل امه عربي
3135,3153,انا مش عم خوفك يا قراع انا عم قلك انقبر انقلع ...,Violent,مش عم خوف يا قراع عم قل انقبر انقلع خلق يا قرد...


## `Model #1`

In [5]:
clean_data = QA_df[["answer_category_num", "question_description_clean"]].copy().\
rename(columns={"question_description_clean": "text", "answer_category_num": "label"})

In [6]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
clean_data["label"] = le.fit_transform(clean_data["label"])
clean_data["label"] = clean_data["label"].astype(np.float32)
clean_data

C:\Users\modaj\anaconda3\envs\DL-torch\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


,label,text
0,3.0,وزير خارجي لبناني جبران باسيل سلسله تغريد عقب ...
1,5.0,سوري بلد حضاره ربط عليه حيوان
2,2.0,قتل وسام حسن وتترحموعليه اي صنف مخلوق انتم
3,1.0,معك نوي بلد قطر متل سمي مساحه اكبر لبنان ل عيب...
4,1.0,امانه قوت موسم ل ي طاف هوا متحمس حق الجاي ان ا...
...,...,...
3132,5.0,كلام هراء قيمه ذكر بلا البلوات منتقل قتل قتيل ...
3133,1.0,خبر شو شايفه جبران باسيل
3134,4.0,كلمه جبران باسيل اخجل اخجل امه عربي
3135,5.0,مش عم خوف يا قراع عم قل انقبر انقلع خلق يا قرد...


In [7]:
# dataset = load_dataset("yelp_review_full")
# dataset

In [8]:
from datasets import Dataset

dataset = Dataset.from_pandas(clean_data).train_test_split(test_size=0.15)

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 2666
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 471
    })
})

In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  6.99ba/s]


In [11]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=1)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [12]:
# import torch

In [13]:
# device = "cuda:0" if torch.cuda.is_available() else "cpu"
# device

In [14]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", num_train_epochs=10,
                                  per_device_train_batch_size=2, per_device_eval_batch_size=2)

In [15]:
train_dataset = tokenized_datasets["train"] 
eval_dataset = tokenized_datasets["test"] 

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
C:\Users\modaj\anaconda3\envs\DL-torch\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2666
  Num Epochs = 10
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 13330


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

## `Model #2`

In [ ]:
f